In [ ]:
!pip install geopandas
!pip install pyshp
!pip install shapely
!pip install geoplot
!pip install altair
!pip install vega
!pip install gpdvega

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.cloud import bigquery
from google.colab import auth

import pandas as pd
import json
import geopandas as gpd
from shapely import wkt
from shapely import geometry
import matplotlib.pyplot as plt
import altair as alt
import gpdvega
import vega
import time

In [ ]:
final_df=pd.read_csv('data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (18,20,21,22,25,28,29,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
final_df.head()

,episode_id,event_id,state,state_fips_code,event_type,cz_type,cz_fips_code,cz_name,wfo,event_begin_time,...,tor_length,tor_width,tor_other_wfo,location_index,event_range,event_azimuth,reference_location,event_latitude,event_longitude,event_point
0,NaN,10081395.0,New hampshire,33.0,hail,C,9.0,GRAFTON,NaN,1957-07-14 13:00:00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,10080147.0,Nevada,32.0,hail,C,31.0,WASHOE,NaN,1957-05-29 18:30:00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,10042631.0,Maine,23.0,hail,C,0.0,NaN,NaN,1957-07-27 20:00:00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,9986168.0,Connecticut,9.0,hail,C,0.0,NaN,NaN,1957-07-26 18:00:00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,9986165.0,Connecticut,9.0,hail,C,9.0,NEW HAVEN,NaN,1957-05-23 21:30:00,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1725003 entries, 0 to 1725002
Data columns (total 33 columns):
 #   Column              Dtype  
---  ------              -----  
 0   episode_id          float64
 1   event_id            float64
 2   state               object 
 3   state_fips_code     float64
 4   event_type          object 
 5   cz_type             object 
 6   cz_fips_code        float64
 7   cz_name             object 
 8   wfo                 object 
 9   event_begin_time    object 
 10  event_timezone      object 
 11  event_end_time      object 
 12  injuries_direct     float64
 13  injuries_indirect   float64
 14  deaths_direct       float64
 15  deaths_indirect     float64
 16  damage_property     float64
 17  damage_crops        float64
 18  source              object 
 19  magnitude           float64
 20  magnitude_type      object 
 21  flood_cause         object 
 22  tor_f_scale         object 
 23  tor_length          float64
 24  tor_width           floa

In [ ]:
final_df['state']=final_df['state']
final_df['event_type']=final_df['event_type']
final_df['event_id']=final_df['event_id'].count()
final_df['damage_property']=final_df['damage_property'].sum()
final_df['damage_crops']=final_df['damage_crops'].sum()
sum_column = final_df["damage_property"] + final_df["damage_crops"]
final_df["damage_cost"] = sum_column

In [ ]:
storm_data_state=pd.read_csv('storm_data_state.csv')

In [ ]:
from vega_datasets import data
states = alt.topo_feature(data.us_10m.url, feature='states')

# Encode basemap to depict all states
basemap= alt.Chart(states).mark_geoshape(
        stroke='white',
        strokeWidth=1,
        fill='lightgray'
    ).properties(
        width=500,
        height=400
    ).project('albersUsa')

# Encode filled map to depict damage cost by states
storm_data_by_state = storm_data_state.groupby(['state','state_fips_code'])['damage_cost'].sum().reset_index(name='damage_cost')
storm_data_by_state['state_fips_code']=storm_data_by_state['state_fips_code']
statemap = alt.Chart(states).mark_geoshape(  

).encode(color=alt.Color('damage_cost:Q', scale=alt.Scale(scheme='greenblue'),
                         legend=alt.Legend(title = "Total damage Cost")),
         tooltip=['state:N','damage_cost:Q']         
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(storm_data_by_state, 'state_fips_code', ['damage_cost','state']) 

)

# Encode point map to depict no. of storms by state
pointmap = alt.Chart(storm_data_state).mark_circle().encode(
    longitude='state_longitude:Q',
    latitude='state_latitude:Q',
    size=alt.Size('sum(no_of_storms):Q', title='Number of Storms'),
    color=alt.Color('sum(no_of_storms):Q', scale=alt.Scale(scheme='blueorange'),
                legend=alt.Legend(orient='right')),
    tooltip=['state:N','sum(no_of_storms):Q', 'sum(damage_cost):Q']
)

costmap = basemap + statemap


/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


In [ ]:
# Layer the filled and point maps
alt.layer(costmap, pointmap).resolve_legend(
    color="independent",
    size="independent"
).resolve_scale(color="independent")

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.LayerChart(...)

In [ ]:
alt.Chart(storm_data_state).transform_aggregate(
    damage = 'sum(damage_cost)',
    groupby = ['event_type']
).transform_window(
    rank='rank(damage)',
    sort=[alt.SortField('damage', order='descending')]
).transform_filter(
    alt.datum.rank <=10
).mark_bar().encode(
    alt.X('damage:Q', title='Total Damage Cost'), 
    alt.Y('event_type:N', sort='-x'),       
    color=alt.value('green'),    
    tooltip = ['event_type:N','sum(no_of_storms):Q','sum(damage_cost):Q']
)

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.Chart(...)

In [ ]:
alt.Chart(storm_data_state).mark_point().encode(
    x='no_of_storms:Q',
    y='damage_cost:Q',
    color='event_type:N',    
    tooltip = ['state:N','event_type:N','no_of_storms:Q','damage_cost:Q']
).configure_point(
    size=100
)

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.Chart(...)